In [ ]:
import chess.pgn
import tqdm
import os

file_name = "100_million_games"
source_file_path = f'../../testData/splitted_games/{file_name}.pgn'
target_folder = '../../testData/split_games/'
os.makedirs(target_folder, exist_ok=True)

def split_pgn_file(source_file, target_folder, num_games_per_file):
    game_count = 0
    file_count = 1
    current_file_path = os.path.join(target_folder, f"{file_name}_part_{file_count}.pgn")

    with open(source_file, 'r') as pgn:
        current_file = open(current_file_path, 'w')
        while True:
            headers = chess.pgn.read_headers(pgn)
            if headers is None:
                break  # End of file

            game = chess.pgn.read_game(pgn)  # Go back and read the entire game
            if game is not None:
                game_count += 1
                print(game, file=current_file, end="\n\n")

                if game_count >= num_games_per_file:
                    current_file.close()
                    file_count += 1
                    game_count = 0
                    current_file_path = os.path.join(target_folder, f"{file_name}_part_{file_count}.pgn")
                    current_file = open(current_file_path, 'w')

        if not current_file.closed:
            current_file.close()

# Customize the number of games per file based on your memory capacity and needs
num_games_per_file = 10000  # Adjust this number based on your observation
split_pgn_file(source_file_path, target_folder, num_games_per_file)


In [3]:
import chess.pgn
import pandas as pd
import tqdm

file_path = '../../testData/splitted_games/100_million_games.pgn'
output_file = '../../testData/splitted_games/processed_moves.csv'
chunk_size = 100000  # Define how many games to process at a time

# Initialize a progress bar
pbar = tqdm.tqdm(total=NUM_GAMES)

# Open the output CSV file in append mode
with open(output_file, 'a', newline='') as csvfile:
    # Initialize a pandas DataFrame to hold chunks of processed games
    chunk_data = []
    
    with open(file_path, 'r') as pgn:
        while True:
            game = chess.pgn.read_game(pgn)
            if game is None:  # End of file
                break
            
            # Extract moves and truncate to the first 30 moves
            moves = [move.uci() for move in game.mainline_moves()][:30]
            chunk_data.append({'moves': str(moves)})
            
            # Check if the chunk size is reached
            if len(chunk_data) >= chunk_size:
                # Convert the chunk data to a DataFrame and append it to the CSV
                df = pd.DataFrame(chunk_data)
                df.to_csv(csvfile, mode='a', header=(csvfile.tell() == 0), index=False)
                chunk_data = []  # Reset the chunk data
            
            pbar.update(1)  # Update the progress bar

    # Process any remaining games
    if chunk_data:
        df = pd.DataFrame(chunk_data)
        df.to_csv(csvfile, mode='a', header=(csvfile.tell() == 0), index=False)

pbar.close()


1772186it [1:54:09, 173.03it/s]                     

KeyboardInterrupt: 

1772195it [1:54:21, 173.03it/s]